# 07-视频生成测试

## CogVideoX-Flash 模型能力

根据官方文档，CogVideoX-Flash 具备以下能力：

| 能力 | 支持情况 | 说明 |
|:---|:---:|:---|
| **视频生成** | ✅ 支持 | 根据文本/图片生成视频 |
| **免费使用** | ✅ 支持 | 智谱免费视频生成模型 |

## 模型规格

| 规格项 | 说明 |
|:---|:---|
| 模型类型 | 视频生成模型 |
| API 类型 | **异步生成** |
| 定价 | **免费** |

## ⚠️ 重要说明

> **CogVideoX-Flash 使用异步生成机制**
> 
> 1. 提交生成任务（返回 task_id）
> 2. 轮询查询任务状态
> 3. 获取生成结果（视频 URL）
> 
> **注意：** 视频生成时间较长，通常需要几十秒到几分钟

## 适用场景

- 短视频创作 - 快速生成创意视频素材
- 广告创意 - 产品宣传视频生成
- 教育培训 - 教学视频制作
- 社交媒体 - 平台内容创作
- 娱乐内容 - 个人创作、艺术创作

## 测试内容
1. 环境配置
2. 提交视频生成任务
3. 查询任务状态
4. 获取生成结果
5. 完整生成流程（提交→轮询→获取结果）
6. 不同场景测试
7. 错误处理

## 1. 环境配置

In [ ]:
# 安装依赖
# !pip install openai python-dotenv requests time

In [ ]:
import os
import time
import json
import requests
from openai import OpenAI

# 加载环境变量
try:
    from dotenv import load_dotenv
    load_dotenv()
    api_key = os.getenv('VITE_ZHIPU_API_KEY')
except:
    api_key = None

# 如果环境变量中没有，使用默认 API Key
if not api_key:
    api_key = "f7ec61a7856a4f4389e037eb588c69df.0pZgcFlTCk9GHCXM"

BASE_URL = "https://open.bigmodel.cn/api/paas/v4"

# 初始化客户端
client = OpenAI(
    api_key=api_key,
    base_url=BASE_URL
)

print("✅ 客户端初始化成功！")
print(f"模型：CogVideoX-Flash")
print(f"类型：视频生成模型")
print(f"生成方式：异步生成")
print(f"⚠️ 注意：视频生成需要较长时间，请耐心等待")

In [ ]:
# 定义视频生成辅助函数
def submit_video_task(prompt, image_url=None, size="1024x1024", fps=30, duration=5):
    """
    提交视频生成任务
    
    Args:
        prompt: 视频描述文本
        image_url: 首帧图片URL（可选）
        size: 分辨率
        fps: 帧率
        duration: 视频时长（秒）
    
    Returns:
        task_id: 任务ID
    """
    try:
        # 构建请求体
        payload = {
            "model": "cogvideox-flash",
            "prompt": prompt,
            "size": size,
            "fps": fps,
            "duration": duration
        }
        
        # 如果提供了首帧图片
        if image_url:
            payload["image_url"] = image_url
        
        # 发送请求
        response = requests.post(
            f"{BASE_URL}/videos/generations",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json"
            },
            json=payload,
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            task_id = result.get("id")
            print(f"✅ 任务提交成功！Task ID: {task_id}")
            return task_id
        else:
            print(f"❌ 提交失败：{response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 提交异常：{e}")
        return None


def query_video_task(task_id):
    """
    查询视频生成任务状态
    
    Args:
        task_id: 任务ID
    
    Returns:
        任务状态信息
    """
    try:
        response = requests.get(
            f"{BASE_URL}/async-result/{task_id}",
            headers={
                "Authorization": f"Bearer {api_key}"
            },
            timeout=30
        )
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ 查询失败：{response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 查询异常：{e}")
        return None


def wait_for_video(task_id, max_wait=300, interval=5):
    """
    等待视频生成完成
    
    Args:
        task_id: 任务ID
        max_wait: 最大等待时间（秒）
        interval: 查询间隔（秒）
    
    Returns:
        生成结果或 None
    """
    start_time = time.time()
    
    print(f"⏳ 开始轮询任务状态（Task ID: {task_id}）...")
    print(f"   最大等待时间：{max_wait}秒")
    print(f"   查询间隔：{interval}秒\n")
    
    while time.time() - start_time < max_wait:
        result = query_video_task(task_id)
        
        if not result:
            print("❌ 查询失败，继续等待...")
            time.sleep(interval)
            continue
        
        status = result.get("status", "unknown")
        elapsed = int(time.time() - start_time)
        
        print(f"[{elapsed}s] 状态：{status}")
        
        if status == "completed":
            print(f"\n✅ 视频生成完成！")
            return result
        elif status == "failed":
            print(f"\n❌ 视频生成失败：{result.get('error', '未知错误')}")
            return None
        elif status in ["processing", "pending"]:
            time.sleep(interval)
        else:
            print(f"   未知状态，继续等待...")
            time.sleep(interval)
    
    print(f"\n⏰ 等待超时（超过 {max_wait} 秒）")
    return None

print("✅ 辅助函数已定义")
print("\n函数说明：")
print("• submit_video_task() - 提交视频生成任务")
print("• query_video_task() - 查询任务状态")
print("• wait_for_video() - 等待视频生成完成")

## 2. 完整视频生成流程

In [ ]:
# 2.1 完整流程测试 - 文生视频
print("🎬 测试：文生视频完整流程\n")
print("=" * 60)

# 定义视频描述
prompt = "一只可爱的小猫在草地上追逐蝴蝶，阳光明媚，微风轻拂，画面温馨治愈"

print(f"📝 视频描述：{prompt}\n")

# 步骤1：提交任务
print("步骤1：提交视频生成任务...")
task_id = submit_video_task(prompt, size="1024x1024", duration=5)

if not task_id:
    print("❌ 任务提交失败，终止测试")
else:
    # 步骤2：等待生成完成
    print(f"\n步骤2：等待视频生成...\n")
    result = wait_for_video(task_id, max_wait=180, interval=5)
    
    if result:
        # 步骤3：获取视频URL
        video_url = result.get("video_url") or result.get("url")
        if video_url:
            print(f"\n✅ 视频URL：{video_url[:80]}...")
            print(f"\n🎬 视频生成成功！可以在浏览器中打开上述URL查看视频")
        else:
            print(f"\n⚠️ 未找到视频URL，完整结果：")
            print(json.dumps(result, indent=2, ensure_ascii=False))
    else:
        print("\n❌ 视频生成失败或超时")

print("\n" + "=" * 60)

## 3. 不同场景测试

In [ ]:
# 3.1 自然风景场景
print("🌅 测试：自然风景场景\n")

prompt = "日出时分的海边，金色的阳光洒在海面上，海浪轻轻拍打着沙滩，几只海鸥飞过天空"

print(f"📝 描述：{prompt}\n")

task_id = submit_video_task(prompt, duration=5)

if task_id:
    result = wait_for_video(task_id, max_wait=180, interval=5)
    if result:
        video_url = result.get("video_url") or result.get("url")
        if video_url:
            print(f"\n✅ 生成成功！")
            print(f"🎬 视频URL：{video_url[:80]}...")

In [ ]:
# 3.2 动物场景
print("\n🐕 测试：动物场景\n")

prompt = "一只金毛犬在公园里奔跑，摇着尾巴，非常开心，背景是绿树和草地"

print(f"📝 描述：{prompt}\n")

task_id = submit_video_task(prompt, duration=5)

if task_id:
    result = wait_for_video(task_id, max_wait=180, interval=5)
    if result:
        video_url = result.get("video_url") or result.get("url")
        if video_url:
            print(f"\n✅ 生成成功！")
            print(f"🎬 视频URL：{video_url[:80]}...")

In [ ]:
# 3.3 城市景观
print("\n🏙️ 测试：城市景观\n")

prompt = "夜晚的城市天际线，高楼大厦灯火辉煌，车流穿梭在街道上，霓虹灯闪烁"

print(f"📝 描述：{prompt}\n")

task_id = submit_video_task(prompt, duration=5)

if task_id:
    result = wait_for_video(task_id, max_wait=180, interval=5)
    if result:
        video_url = result.get("video_url") or result.get("url")
        if video_url:
            print(f"\n✅ 生成成功！")
            print(f"🎬 视频URL：{video_url[:80]}...")

## 4. 图生视频（首帧图片）

In [ ]:
# 4.1 基于首帧图片生成视频
print("🖼️➡️🎬 测试：图生视频\n")

# 使用一张图片作为首帧
image_url = "https://cdn.bigmodel.cn/static/logo/register.png"
prompt = "基于这张图片，展示动态效果，画面轻微晃动，有呼吸感"

print(f"🖼️ 首帧图片：{image_url}")
print(f"📝 描述：{prompt}\n")

task_id = submit_video_task(prompt, image_url=image_url, duration=5)

if task_id:
    result = wait_for_video(task_id, max_wait=180, interval=5)
    if result:
        video_url = result.get("video_url") or result.get("url")
        if video_url:
            print(f"\n✅ 图生视频成功！")
            print(f"🎬 视频URL：{video_url[:80]}...")

## 5. 批量生成测试

In [ ]:
# 5.1 批量提交多个视频任务
print("📦 测试：批量生成多个视频\n")

prompts = [
    "樱花飘落的公园小道，微风轻拂，花瓣飞舞，浪漫氛围",
    "雨后的城市街道，地面有积水倒影，行人撑伞走过",
    "雪山脚下的湖泊，水面平静如镜，倒映着蓝天白云"
]

task_ids = []

# 提交所有任务
print("步骤1：批量提交任务...\n")
for i, prompt in enumerate(prompts, 1):
    print(f"任务{i}：{prompt[:30]}...")
    task_id = submit_video_task(prompt, duration=5)
    if task_id:
        task_ids.append((i, task_id))
    print()

print(f"✅ 成功提交 {len(task_ids)}/{len(prompts)} 个任务\n")

# 等待所有任务完成
print("步骤2：等待所有任务完成...\n")
results = []
for idx, task_id in task_ids:
    print(f"\n处理任务 {idx}...")
    result = wait_for_video(task_id, max_wait=180, interval=5)
    if result:
        video_url = result.get("video_url") or result.get("url")
        if video_url:
            results.append((idx, video_url))
            print(f"✅ 任务 {idx} 完成")
        else:
            print(f"⚠️ 任务 {idx} 完成但未获取到URL")
    else:
        print(f"❌ 任务 {idx} 失败")

# 汇总结果
print(f"\n{'='*60}")
print(f"📊 批量生成结果：{len(results)}/{len(task_ids)} 成功")
for idx, url in results:
    print(f"  任务{idx}：{url[:60]}...")

## 6. 错误处理测试

In [ ]:
# 6.1 测试：空提示词
print("⚠️ 测试：空提示词\n")

try:
    task_id = submit_video_task("")
    if task_id:
        print("⚠️ 任务提交成功（可能使用默认提示词）")
    else:
        print("✅ 正确拒绝空提示词")
except Exception as e:
    print(f"✅ 错误捕获：{type(e).__name__}")
    print(f"   错误：{str(e)[:100]}...")

In [ ]:
# 6.2 测试：无效的任务ID
print("\n⚠️ 测试：无效的任务ID\n")

result = query_video_task("invalid-task-id-12345")
if result:
    print(f"结果：{result}")
else:
    print("✅ 正确处理无效任务ID")

In [ ]:
# 6.3 测试：超长提示词
print("\n⚠️ 测试：超长提示词\n")

long_prompt = "一只可爱的小猫在草地上玩耍，" * 50  # 很长的提示词

try:
    task_id = submit_video_task(long_prompt, duration=5)
    if task_id:
        print(f"✅ 超长提示词处理成功，任务ID：{task_id}")
        # 取消任务（不等待完成）
        print("（已提交但不等待完成，仅测试提交流程）")
except Exception as e:
    print(f"✅ 错误捕获：{type(e).__name__}")
    print(f"   错误：{str(e)[:100]}...")

## 7. 总结

In [ ]:
print("""
╔════════════════════════════════════════════════════════════════╗
║                CogVideoX-Flash 测试总结                        ║
╠════════════════════════════════════════════════════════════════╣
║ 模型定位：免费视频生成模型                                       ║
╠════════════════════════════════════════════════════════════════╣
║ 模型规格：                                                      ║
║   • 类型：视频生成模型                                          ║
║   • 生成方式：异步生成                                          ║
║   • 定价：**免费**                                              ║
╠════════════════════════════════════════════════════════════════╣
║ 生成流程：                                                      ║
║   1️⃣ 提交任务 → 获取 Task ID                                    ║
║   2️⃣ 轮询查询 → 获取任务状态                                    ║
║   3️⃣ 生成完成 → 获取视频 URL                                    ║
╠════════════════════════════════════════════════════════════════╣
║ 支持的功能：                                                    ║
║   • 文生视频 - 根据文本描述生成视频                             ║
║   • 图生视频 - 基于首帧图片生成视频                             ║
║   • 批量生成 - 同时提交多个任务                                 ║
╠════════════════════════════════════════════════════════════════╣
║ 适用场景：                                                      ║
║   • 短视频创作 - 快速生成创意视频素材                           ║
║   • 广告创意 - 产品宣传视频生成                                 ║
║   • 教育培训 - 教学视频制作                                     ║
║   • 社交媒体 - 平台内容创作                                     ║
║   • 娱乐内容 - 个人创作、艺术创作                               ║
╠════════════════════════════════════════════════════════════════╣
║ 注意事项：                                                      ║
║   ⚠️ 视频生成需要较长时间（几十秒到几分钟）                      ║
║   ⚠️ 使用异步生成机制，需要轮询查询状态                          ║
║   ⚠️ 不是对话模型，不能使用 chat.completions                    ║
╚════════════════════════════════════════════════════════════════╝
""")